In [1]:
import math
import numpy as np
import scipy as sp
from itertools import filterfalse
from itertools import combinations

In [2]:
def calculate_tstar(n11, n01, n, N):
    """blurb here.
    """

    return n11/n - n01/(N-n)

In [4]:
def calculate_tN(N01, N10, N):
    """blurb here."""
    return (N01 - N10) / N

In [5]:
def filterTable(Nt, n00, n01, n10, n11):
    '''
    check whether summary table Nt of binary outcomes is consistent with observed counts
    
    implements the test in Theorem 1 of Li and Ding (2016)
    
    Parameters:
    ----------
    Nt : list of four ints
        the table of counts of subjects with each combination of potential outcomes, in the order
        N_00, N_01, N_10, N_11
       
    n01 : int
        number of subjects assigned to control whose observed response was 1

    n11 : int
        number of subjects assigned to treatment whose observed response was 1
        
    Returns:
    --------
    ok : boolean
        True if table is consistent with the data
    '''
    N = np.sum(Nt)   # total subjects
    return max(0,n11-Nt[2], Nt[3]-n01, Nt[2]+Nt[3]-n10-n01) <= min(Nt[3], n11, Nt[2]+Nt[3]-n01, N-Nt[2]-n01-n10)

In [6]:
def N_generator(N, n00, n01, n10, n11):
    ''' 
    generate tables algebraically consistent with data from an experiment with binary outcomes
    
    Parameters
    ----------
    N : int
        number of subjects
    n00 : int
        number of subjects assigned to treatment 0 who had outcome 0
    n01 : int
        number of subjects assigned to treatment 0 who had outcome 0
    n10 : int
        number of subjects assigned to treatment 1 who had outcome 0
    n11 : int
        number of subjects assigned to treatment 1 who had outcome 1
    
    Returns
    -------
    Nt : list of 4 ints 
        N00, subjects with potential outcome 0 under treatments 0 and 1
        N01, subjects with potential outcome 0 under treatment 0 and 1 under treatment 1
        N10, subjects with potential outcome 1 under treatment 0 and 0 under treatment 1
        N11, subjects with potential outcome 1 under treatments 0 and 1
    '''
    for i in range(min(N-n00, N-n10)+1):               # allocate space for the observed 0 outcomes, n00 and n10
        N11 = i                                           
        for j in range(max(0, n01-N11), N-n00-N11):    # N11+N10 >= n01; N11+N10+n00 <= N
            N10 = j                                        
            for k in range(max(0, n11-N11), min(N-n10-N11, N-N11-N10)): 
                                                       # N11+N01 >= n11; N11+N01+n10 <= N; no more than N subjects
                N01 = k                                  
                N00 = N-N11-N10-N01                  
                if filterTable([N00, N01, N10, N11], n00, n01, n10, n11):
                    yield [N00, N01, N10, N11]
                else:
                    pass

In [7]:
def potential_outcomes(Nt):
    '''
    make a 2xN table of potential outcomes from the 2x2 summary table Nt
    
    Parameters
    ----------   
    Nt : list of 4 ints
        N00, N01, N10, N11
    
    Returns
    -------
    po : Nx2 table of potential outcomes consistent with Nt
    '''
    return np.reshape(np.array([0,0]*Nt[0]+[0,1]*Nt[1]+[1,0]*Nt[2]+[1,1]*Nt[3]), [-1,2])

In [8]:
# original tau_twosided we made
def tau_twosided_ci(n11, n10, n01, n00, alpha):
    """blurb here"""
    N = n11 + n10 + n01 + n00
    n = n10 + n11
    generate = [Nt for Nt in N_generator(N, n00, n01, n10, n11)]
    tN = []
    tau_star = calculate_tstar(n11, n01, n, N)
    
    confidence_set = []
    arr = []
    
    for i in generate:
        tN = calculate_tN(i[1], i[2], N)
        t = abs(tau_star - tN)
        rows = potential_outcomes(i)
        stats = []
        # exact = True
        combs = combinations(rows, n)
#         if combos >= max_combinations:
        for j in combs:
            x_0 = np.sum(j, axis = 0)[0]
            x_1 = np.sum(j, axis = 0)[1]
            tau_hat = (x_1/n) - (x_0/(N-n))
            stat = abs(tau_hat - tN)
            stats.append(stat)
#             print(stat)
        maximum = np.percentile(stats, 100*(1-alpha))
        
        if t > maximum:
            pass
        else:
            confidence_set.append(tN)
            
    ci_upper = max(confidence_set)
    ci_lower = min(confidence_set)
    return np.array([ci_lower, ci_upper])
    


In [9]:
tau_twosided_ci(1, 1, 1, 13, .05) * 16

array([-1., 13.])

In [24]:
tau_twosided_ci(1, 1, 3, 19, .05) * 24

array([-3., 14.])

In [16]:
tau_twosided_ci(2, 6,8,0, .05) * 16

ValueError: max() arg is an empty sequence

In [ ]:
tau_twosided_ci(8,4,5,7, .05) * 24

In [ ]:
# Hypothetical experiment
N = 10
n = 5
n00 = 3
n01 = 2
n10 = 1
n11 = 4
[Nt for Nt in N_generator(N, n00, n01, n10, n11)]


In [ ]:
# test
Nt = [5, 4, 3, 2]
p = potential_outcomes(Nt)
p

In [83]:
n=753
m=752
N=n+m
n01 = 59
n11 = 11
n00 = m-n01
n10 = n-n11

In [ ]:
tau_twosided_ci(n11, n10, n01, n00, .05)

In [12]:
tau_twosided_ci(1, 1, 1, 13, .05) * 16

array([-1., 11.])